# Callibration plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# import ..src.binary_classifier as binary_classifier
from ..src import binary_classifier as binary_classifier

: 

In [ ]:
# create some data
n_samples = 1000
y_prob = np.random.uniform(0, 1, n_samples)   # the probability of class 1 predictions
y_true = np.random.choice([0,1], n_samples)   # the actually corresponding class labels

# create figure
fig_auroc = binary_classifier.plot_calibration_curve(y_prob, y_true, save_fig_path=None)

# customize figure
axes = fig_auroc.get_axes()
ax0 = axes[0]
ax0.set_title('New Title for Calibration Plot')

# save figure
fig.savefig('calibration_plot.png', bbox_inches='tight')

: 